In [1]:
import os
import sys
import pickle
initdir = os.getcwd()+'/' 
libz_ = './../../libs/'
sys.path.append('./../../libs/')
os.chdir(initdir+libz_)
opt_order = pickle.load(open('order.dmp','rb'))
true_order = pickle.load(open('true_order.dmp','rb'))
sz_lvl = pickle.load(open('trv.dmp','rb'))
pass_info = pickle.load(open('pass_struct.dmp','rb'))
from sql_connection import mysql_connection
from sel_feature import diff_pass
from make import opts_lvl
from tqdm import tqdm
os.chdir(initdir)
large_dir = initdir+"./../../../Unix_Source/unix_cmp/"
small_dir = initdir+"./../../../Unix_Source/unix_small_2/"
opts = ['opt_O%s' % i for i in ['1','2','3','s','z']]
prv_ind = {'opt_O2': 25, 'opt_O3': 8, 'opt_Os': 105, 'opt_Oz': 297, 'opt_O1': -1}
s_ind = {'opt_O1': 2, 'opt_O2': 97, 'opt_O3': 157, 'opt_Os': 263, 'opt_Oz': 373}
import re
from functools import reduce
import pandas as pd
mflags = ['nnan','ninf','nsz','arcp','contract','afn','reassoc','fast']

In [2]:
class Source:
    signals = []
    def printt(sig,src):
        if sig not in Source.signals:
            print(src)

In [7]:
pass_structure = pd.DataFrame(columns=['name','synt','args','desc','pttrns','opt','gp','num','eq'])
for group in pass_info['pass-info']:
    for inst,det in pass_info['pass-info'][group].items():
        pass_structure = pass_structure.append({'name':inst,'synt':det['synt'],'args':det['args'],'desc':det['overview'],'gp':group,'num':det['synt'].__len__(),'eq':reduce(lambda x,y:x or y,list(map(lambda x: '=' in x,det['synt'])),False)}
                                               ,ignore_index=True)
pass_structure = pass_structure.set_index('name')
del pass_structure.index.name

for pass_ in pass_info['pass-group']:
    pass_structure.loc[pass_]['opt'] = pass_info['pass-group'][pass_]['opt']
    pass_structure.loc[pass_]['pttrns'] = pass_info['pass-group'][pass_]['ptr']
pickle.dump(pass_structure,open(initdir+libz_+'pass_fullinfo.dmp','wb'))

In [8]:
pass_structure = pickle.load(open(initdir+libz_+'pass_fullinfo.dmp','rb'))
ac_pass = list(map(lambda x:x.split('-')[0],pass_structure.index.tolist()))

## Detecting pass id

In [21]:
id_passes = {}
conn = mysql_connection(db_='files')
cursor = conn.cursor()
try:
    for lvl in true_order:
        id_passes[lvl] = {}
        for option in true_order[lvl]['order']:
            cursor.execute("SELECT ID FROM OPTS_LVL WHERE NAME='%s' AND OPT_ID=(SELECT ID FROM OPTIONS WHERE NAME='%s')" %(lvl,option))
            id_passes[lvl][option] = cursor.fetchone()[0]
except Exception as e:
    print(e)
    conn.close()
finally:
    conn.close()
pickle.dump(id_passes,open(initdir+libz_+'id_passes_db','wb'))

In [9]:
id_passes = pickle.load(open(initdir+libz_+'id_passes_db','rb'))

In [8]:
diff_ss = {}
for file in tqdm(os.listdir(small_dir)):
    conn = mysql_connection(db_='files')
    cursor = conn.cursor()
    if os.path.isdir(small_dir+file):
        diff_ss[file] = {}
        c_file = {}
        cursor.execute("SELECT ID FROM TARGET WHERE PG_ID = (SELECT ID FROM PG WHERE PG_NAME='%s')" %file)
        targ_id = cursor.fetchone()[0]
        cursor.execute("SELECT LL_ID FROM RL_LL WHERE TARGET_ID=%s" %(targ_id))
        ll_id = cursor.fetchone()[0]
        cursor.execute("SELECT DATA FROM ETALON_DATA WHERE RL_LL_ID=%s" %(ll_id))
        c_file[-1] = {'file':cursor.fetchone()[0],'size':0}
        for lvl_opt in opt_order:
            for num in true_order[lvl_opt]['order']:
                cursor.execute("SELECT ID FROM OPTS_LVL WHERE NAME='%s' AND OPT_ID=(SELECT ID FROM OPTIONS WHERE NAME='%s')" %(lvl_opt,num))
                id_pass = cursor.fetchone()[0]
                cursor.execute("SELECT SIZE FROM TARGET_RES WHERE TARGET_ID=%s AND OPT_LVL_ID=%s" %(targ_id,id_pass))
                c_file[id_pass] ={'size':cursor.fetchone()[0]}
                cursor.execute("SELECT APPLIED_DATA FROM OPTS_DATA WHERE DATA_ID = (SELECT DATA_ID FROM MAIN_QUERY WHERE RL_LL_ID=%s and OPT_LVL_ID=%s)" %(ll_id,id_pass)) 
                c_file[id_pass] = {'file':cursor.fetchone()[0]}
        p_c = 2
        diff_ss[file]['diff'] = {}
        diff_ss[file]['reply'] = {}
        for lvl_opt in opts_lvl:
            c_file[s_ind[lvl_opt]-1] = c_file[prv_ind[lvl_opt]]
            p_c += sz_lvl[lvl_opt]['size'] 
            for _opt in (true_order[lvl_opt]['order']):
                diff_ss[file]['diff'][id_passes[lvl_opt][_opt]],diff_ss[file]['reply'][id_passes[lvl_opt][_opt]] = diff_pass(c_file[p_c-1]['file'].splitlines(),c_file[p_c]['file'].splitlines(),str(p_c-1),str(p_c))
                p_c+= 1

pickle.dump(diff_ss,open(initdir+'sm_dir_fts.dmp','wb'))       

100%|██████████| 21/21 [00:08<00:00,  2.56it/s]


In [10]:
diff_ss = pickle.load(open(initdir+'sm_dir_fts.dmp','rb'))

In [11]:
Source.signals = [8015,8153,3689,5921,9010]

In [12]:
for file in diff_ss:
    rep_ = diff_ss[file]['reply']
    for opt_lvl in rep_:
        xs = rep_[opt_lvl]
        for ind in range(len(xs)):
            print(ind,xs[ind])
        break
    break
        

0 - 1 --- 

1 + 1 +++ 

2 @@ -1,4 +1,4 @@

3 - 2 -; ModuleID = 'CPT234P3.cpp'
4 + 2 +; ModuleID = 'CPT234P3.bc'
5 a:3 b:3  source_filename = "CPT234P3.cpp"
6 a:4 b:4  target datalayout = "e-m:e-p270:32:32-p271:32:32-p272:64:64-i64:64-f80:128-n8:16:32:64-S128"
7 a:5 b:5  target triple = "x86_64-unknown-linux-gnu"
8 @@ -79,33 +79,23 @@

9 a:80 b:80  define dso_local i32 @_Z8GetCountv() #0 {
10 a:81 b:81  entry:
11 a:82 b:82    %Count = alloca i32, align 4
12 - 83 -  %GreaterThen1 = alloca i32, align 4
13 a:84 b:83    %call = call dereferenceable(8) %"class.std::basic_ostream"* @_ZStlsISt11char_traitsIcEERSt13basic_ostreamIcT_ES5_PKc(%"class.std::basic_ostream"* dereferenceable(8) @_ZSt4cout, i8* getelementptr inbounds ([24 x i8], [24 x i8]* @.str.4, i64 0, i64 0))
14 a:85 b:84    %call1 = call dereferenceable(16) %"class.std::basic_istream"* @_ZNSirsERi(%"class.std::basic_istream"* @_ZSt3cin, i32* dereferenceable(4) %Count)
15 a:86 b:85    br label %do.body
16 a:87 b:86  
17 - 88 -do.bod

In [ ]:
def map_pass(pass_,str_pss_,table):
    pd.DataFrame(columns=['var','type','=','pname','в_arg','allign','flags'])
    for ptr in table[pass_]['pttrns']:
        if ptr.__len__() == len(str_pss_):
            print(ptr,str_pss_)
    else:
        for opt in table[pass_]['opt']:
    

In [14]:
ps_ = set()
none_info = set()
for file in diff_ss:
    rep_ = diff_ss[file]['reply']
    block = 0
    for opt_lvl in rep_:
        xs = rep_[opt_lvl]
        ind = 2
        fu_name = ''
        args = ''
        while ind<len(xs):
            ##find function area
            if 'define' in xs[ind] and '{' in xs[ind]:
                Source.printt(9010,xs[ind])
                args = re.split(r'\@.+(?=\()',xs[ind])[1]
                fu_name = re.search(r'\@.+(?=\()',xs[ind])[0]
                args = args[:re.search(r"\)(?:.(?!\)))+\{$",args).span()[0]+1]
            if reduce(lambda x,y : x or y,list(map(lambda x:x in xs[ind][0],['-','+'])),False):
                Source.printt(9010,'Block:{},function:{},args:{}'.format(block,fu_name,args))
                while True:
                    if ind>=len(xs):
                        break
                    if not reduce(lambda x,y : x or y,list(map(lambda x:x in xs[ind][0],['-','+'])),False):
                        block+=1
                        break
                    if (('do.' in xs[ind]) or ('if.' in xs[ind])) and re.match(r'.+\:\;*.*',xs[ind]):
                        Source.printt(8015,xs[ind])
                        ind+=1
                        continue
                    for symb in ['-','+']:
                        if symb==xs[ind][0]:
                            p_str = xs[ind]
                            w_str = xs[ind][2:].split(symb)[1]
                            if type(w_str)==list:
                                w_str = symb.join(w_str)
                                Source.printt(5921,w_str)
                            w_str = re.sub(r'\s{2,100}','',w_str)
                            break
                    if '=' in w_str :
                        w_str = re.split(r'\s\=*\s*',w_str)
                        tmp = w_str[0]
                        w_str[0] = w_str[1]
                        w_str[1] = tmp
                    else:
                        w_str = re.split(r'\s\=*\s*',w_str)  
                    if w_str[0] not in ac_pass and ' '.join(w_str)!='':
                        Source.printt(8153,'None informative {} strings:{},old:{}'.format(ind,w_str,p_str))
                        none_info.add(p_str)
                    elif ' '.join(w_str)!='':
                        ps_.add(w_str[0])
#                         Source.printt(6391,w_str[0])
                    ind+=1
            if ind<len(xs) and '}' in xs[ind]:
                Source.printt(9010,xs[ind])
            ind+=1                          

        

In [15]:
search_index = []
for pass_ in ps_:
    if pass_+'-instruction' in pass_structure.index:
        search_index.append(pass_+'-instruction')
    elif pass_+'-to-instruction' in pass_structure.index:
        search_index.append(pass_+'-to-instruction')
    else:
        print('Homie')


In [98]:
def bbcmd_parser(op,math_flags=mflags):
    pd_ser = {}
    flags_ = []
    pd_ser['flags'] = flags_
    pd_ser['name'] = op[0]
    pd_ser['res'] = op[1]
    op = op[2:]
    if ('nsw' in op) or ('nuw' in op) or ('react' in op):
        if 'nsw' in op and 'nuw' in op:
            pd_ser['d_flags'] = op[:2]
            op = op[2:]
        else:
            pd_ser['d_flags'] = [op[0]]
            op = op[1:]
    while reduce(lambda x,y : x or y,list(map(lambda x: x in op,math_flags)),False):
        flags.append(op[0])
        op = op[1:]
    pd_ser['ty'] = op[0]
    pd_ser['op_1'] = op[1]
    pd_ser['op_2'] = op[2]
    if '%' in pd_ser['op_1']:
        val_1 = True
    else:
        val_1 = False
    if '%' in pd_ser['op_2']:
        val_2 = True
    else:
        val_2 = False
    pd_ser['is_val_2'] = val_2
    pd_ser['is_val_1'] = val_1
    return pd.Series(pd_ser,index=['res','name','flags','ty','is_val_1','is_val_2','op_1','op_2','d_flags'])
        
        
    
        

In [116]:
def concmd_parser(op):
    pd_ser = {}
    pd_ser['name'] = op[0]
    pd_ser['res'] = op[1]
    pd_ser['ty'] = op[2]
    pd_ser['val'] = op[3]
    pd_ser['ty2'] = op[5]
    return pd.Series(pd_ser,index=['name', 'res', 'ty', 'val', 'ty2'])

In [ ]:
def agcmd_parser(op):
    op =op.reverse()
    pd_ser={}
    pd_ser['name'] = op.pop()
    pd_ser['res'] = op.pop()
    pd_ser['own_ty'] = op.pop()
    pd_ser['val'] = op.pop()
    if 'insert' in pd_ser['name']:
        pd_ser['ty2'] = op.pop()
        pd_ser['ty3'] = op.pop()
    while op!=[]    
        pd_ser['idx'] = op[0]

In [126]:
get_ptrns('vector-operations')

['extractelement', '<result>', '<n', 'x', '<ty>>', '<val>,', '<ty2>', '<idx>']
['extractelement', '<result>', '<vscale', 'x', 'n', 'x', '<ty>>', '<val>,', '<ty2>', '<idx>']
['insertelement', '<result>', '<n', 'x', '<ty>>', '<val>,', '<ty>', '<elt>,', '<ty2>', '<idx>']
['insertelement', '<result>', '<vscale', 'x', 'n', 'x', '<ty>>', '<val>,', '<ty>', '<elt>,', '<ty2>', '<idx>']
['shufflevector', '<result>', '<n', 'x', '<ty>>', '<v1>,', '<n', 'x', '<ty>>', '<v2>,', '<m', 'x', 'i32>', '<mask>']
['shufflevector', '<result>', '<vscale', 'x', 'n', 'x', '<ty>>', '<v1>,', '<vscale', 'x', 'n', 'x', '<ty>>', 'v2,', '<vscale', 'x', 'm', 'x', 'i32>', '<mask>']


In [88]:
def get_ptrns(group_):
    for instr_ in pass_structure[(pass_structure.gp ==group_)].pttrns:
        for ptrn in instr_:
            print(ptrn)

In [56]:
for ptrns in pass_structure[(pass_structure.gp =='terminator-instructions')].pttrns:
    for ptr in ptrns:
        if reduce(lambda x,y : x or y,list(map(lambda x: x in term_inst.keys(),ptr[1:])),False):
            print(ptr)

['ret', 'void']
['br', 'i1', '<cond>,', 'label', '<iftrue>,', 'label', '<iffalse>']
['br', 'label', '<dest>']
['switch', '<intty>', '<value>,', 'label', '<defaultdest>', '']
['', 'to', 'label', '<normal', 'label>', 'unwind', 'label', '<exception', 'label>']
['', 'to', 'label', '<fallthrough', 'label>', '']
['catchswitch', '<resultval>', 'within', '<parent>', 'unwind', 'label', '<default>']
['catchret', 'from', '<token>', 'to', 'label', '<normal>']
['cleanupret', 'from', '<value>', 'unwind', 'label', '<continue>']
